In [1]:
%pip install llama-index-agent-openai
%pip install llama-index-llms-openai
!pip install "tracing-auto-instrumentation[llama-index]" --upgrade

In [2]:
import llama_index.core

from tracing_auto_instrumentation.llama_index import LlamaIndexCallbackHandler
llama_index.core.global_handler = LlamaIndexCallbackHandler(
    project_name="Function call demo",
)

In [3]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool

import os
import dotenv
from getpass import getpass

dotenv.load_dotenv()
if os.getenv("OPENAI_API_KEY") is None:
    os.environ["OPENAI_API_KEY"] = getpass(
        "Paste your OpenAI key from:"
        " https://platform.openai.com/account/api-keys\n"
    )
assert os.getenv("OPENAI_API_KEY", "").startswith(
    "sk-"
), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")

OpenAI API key configured


In [4]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

llm = OpenAI(model="gpt-3.5-turbo-1106")
agent = OpenAIAgent.from_tools(
    [multiply_tool, add_tool], llm=llm, verbose=True
)

In [5]:
# Note: if you have a APIConnectionError and are on a Mac Silicon device,
# you may need to run the following command in your terminal:
# bash /Applications/Python*/Install\ Certificates.command

# After running the command above, please restart this notebook and try again

response = agent.chat("What is (121 * 3) + 42?")
print(str(response))

Added user message to memory: What is (121 * 3) + 42?


=== Calling Function ===
Calling function: multiply with args: {"a": 121, "b": 3}
Got output: 363

=== Calling Function ===
Calling function: add with args: {"a": 363, "b": 42}
Got output: 405



The result of (121 * 3) is 363, and when you add 42 to it, you get 405.
